---
title: "PS5"
author: "Shreya Shravini"
date: "11/10/2024"
format: 
  pdf:
    include-in-header: 
       text: |
         \usepackage{fvextra}
         \DefineVerbatimEnvironment{Highlighting}{Verbatim}{breaklines,commandchars=\\\{\}}
include-before-body:
  text: |
    \RecustomVerbatimEnvironment{verbatim}{Verbatim}{
      showspaces = false,
      showtabs = false,
      breaksymbolleft={},
      breaklines
    }
output:
  echo: false
  eval: false
---


github link: https://github.com/shreyashravini/PS5

\newpage


In [ ]:
import pandas as pd
import altair as alt
import time

import warnings 
warnings.filterwarnings('ignore')
alt.renderers.enable("png")

## Step 1: Develop initial scraper and crawler

### 1. Scraping (PARTNER 1)


In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

# Define the URL of the HHS OIG Enforcement Actions page
url = "https://oig.hhs.gov/fraud/enforcement/"

# Send a request to retrieve the HTML content
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# Filter out list items that match the specified class for enforcement actions
enforcement_items = soup.find_all("li", class_="usa-card card--list pep-card--minimal mobile:grid-col-12")

# Extract titles of enforcement actions
action_titles = [item.find("h2", class_="usa-card__heading").find("a").get_text(strip=True) for item in enforcement_items]

# Extract dates associated with each action
action_dates = [item.find(class_="text-base-dark padding-right-105").get_text(strip=True) for item in enforcement_items]

# Extract categories for each enforcement action
action_categories = [item.find("ul", class_="display-inline add-list-reset").find("li").get_text(strip=True) for item in enforcement_items]

# Extract links for each enforcement action
action_links = [item.find("a")['href'] for item in enforcement_items if item.find("a")]

# Organize extracted data into a DataFrame
data = {
    "Title": action_titles,
    "Date": action_dates,
    "Category": action_categories,
    "Link": action_links
}

enforcement_df = pd.DataFrame(data)

# Display the first few rows of the DataFrame
enforcement_df.head()


### 2. Crawling (PARTNER 1)


In [ ]:
from urllib.parse import urljoin
import pandas as pd
from bs4 import BeautifulSoup
import requests

# Define the base URL for the HHS OIG Enforcement Actions page
base_url = "https://oig.hhs.gov/fraud/enforcement/"

# Convert relative links to absolute URLs
full_urls = [urljoin(base_url, link) for link in action_links]
print(full_urls)

# Initialize a list to store agency names
agency_names = []

# Loop over each URL to access the page and retrieve the agency name
for url in full_urls:
    response = requests.get(url)
    page_soup = BeautifulSoup(response.text, "html.parser")
    
    # Locate the specific <ul> tag containing the agency information
    agency_section = page_soup.find_all("ul", class_="usa-list usa-list--unstyled margin-y-2")
    
    for ul in agency_section:
        li_tags = ul.find_all("li")
        
        # Ensure at least two list items are available, then extract the second item
        if len(li_tags) > 1:
            agency_text = li_tags[1].get_text(strip=True).replace("Agency:", "").strip()
            agency_names.append(agency_text)

print(agency_names)

# Create a dictionary to add the agency information to the DataFrame
agency_data = {"Agency": agency_names}

# Convert the dictionary to a DataFrame and concatenate with the original DataFrame
updated_df = pd.concat([enforcement_df, pd.DataFrame(agency_data)], axis=1)

# Display the first few rows of the updated DataFrame
updated_df.head()

## Step 2: Making the scraper dynamic

### 1. Turning the scraper into a function 

* a. Pseudo-Code (PARTNER 2)

Define fetch_page(session, url)
We will fetch and return the HTML content from the url.

Define scrape_action_page(session, link)
We will fetch the enforcement action page and extract the agency name (return "N/A" if not found).

Define scrape_enforcement_actions(year, month)
We will check if the year is before 2013 and stop if true.
We will initialize variables, set the target date, and create an asynchronous session.
We will loop through pages, fetch HTML, extract enforcement data, and queue tasks to scrape agency names.
We will store data for actions from the target date and continue until no entries are found.
We will return the collected data as a DataFrame.

Define main()
We will prompt for the year and month, call scrape_enforcement_actions, display the DataFrame, and save it as a CSV.

Run the script
We will execute the main() function with asyncio.run().

* b. Create Dynamic Scraper (PARTNER 2)


In [ ]:
import asyncio
import aiohttp
import nest_asyncio
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin

async def fetch_page(session, url):
    async with session.get(url) as response:
        return await response.text()

async def extract_agency_info(session, action_url):
    html = await fetch_page(session, action_url)
    soup = BeautifulSoup(html, "html.parser")
    agency_list = soup.find("ul", class_="usa-list usa-list--unstyled margin-y-2")
    
    if agency_list and len(agency_list.find_all("li")) > 1:
        agency = agency_list.find_all("li")[1].get_text(strip=True).replace("Agency:", "").strip()
    else:
        agency = "N/A"
    return agency

async def scrape_enforcement_data(year, month):
    if year < 2013:
        print("Please input a year >= 2013. Enforcement actions are listed only from 2013 onwards.")
        return None

    base_url = "https://oig.hhs.gov/fraud/enforcement/"
    page_number = 1
    
    titles, dates, categories, links, agencies = [], [], [], [], []
    start_date = datetime(year, month, 1).date()
    
    connector = aiohttp.TCPConnector(ssl=False)
    async with aiohttp.ClientSession(connector=connector) as session:
        while True:
            current_page_url = f"{base_url}?page={page_number}" if page_number > 1 else base_url
            print(f"Scraping: {current_page_url}")
            
            html = await fetch_page(session, current_page_url)
            soup = BeautifulSoup(html, "html.parser")
            
            enforcement_entries = soup.find_all("li", class_="usa-card card--list pep-card--minimal mobile:grid-col-12")
            
            if not enforcement_entries:
                print(f"No more entries found on page {page_number}. Exiting.")
                break
            
            action_tasks = []
            for entry in enforcement_entries:
                date_str = entry.find(class_="text-base-dark padding-right-105").get_text(strip=True)
                entry_date = datetime.strptime(date_str, "%B %d, %Y").date()
                
                if entry_date >= start_date:
                    title = entry.find("h2", class_="usa-card__heading").find("a").get_text(strip=True)
                    category = entry.find("ul", class_="display-inline add-list-reset").find("li").get_text(strip=True)
                    link = urljoin(base_url, entry.find("a")["href"])
                    
                    titles.append(title)
                    dates.append(date_str)
                    categories.append(category)
                    links.append(link)
                    
                    action_tasks.append(extract_agency_info(session, link))
                else:
                    agencies.extend(await asyncio.gather(*action_tasks))
                    return pd.DataFrame({
                        "Title of Enforcement Action": titles,
                        "Date": dates,
                        "Category": categories,
                        "Link": links,
                        "Agency": agencies
                    })
            
            agencies.extend(await asyncio.gather(*action_tasks))
            page_number += 1
            await asyncio.sleep(1)
    
    return pd.DataFrame({
        "Title of Enforcement Action": titles,
        "Date": dates,
        "Category": categories,
        "Link": links,
        "Agency": agencies
    })

async def main():
    year, month = 2023, 1 
    df_jan_2023 = await scrape_enforcement_data(year, month)
    print(df_jan_2023)
    
    # Save to CSV
    filename = f"enforcement_actions_{year}_{month:02d}.csv"
    df_jan_2023.to_csv(filename, index=False)
    print(f"Data saved to {filename}")
    
    return df_jan_2023

# Running the script
nest_asyncio.apply()
if __name__ == "__main__":
    df_jan_2023 = asyncio.run(main())

In [ ]:
df_jan_2023.head()

In [ ]:
print(f"The total number of enforcement actions collected is {df_jan_2023.shape[0]}.")

# Details of the first scraped enforcement action (earliest)
first_entry = df_jan_2023.iloc[0].to_dict()
print(first_entry)

* c. Test Partner's Code (PARTNER 1)


In [ ]:
import asyncio
import aiohttp
import nest_asyncio
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin

async def fetch_page_content(session, url):
    async with session.get(url) as response:
        return await response.text()

async def extract_agency_info(session, action_url):
    html = await fetch_page_content(session, action_url)
    soup = BeautifulSoup(html, "html.parser")
    agency_list = soup.find("ul", class_="usa-list usa-list--unstyled margin-y-2")
    
    if agency_list and len(agency_list.find_all("li")) > 1:
        agency = agency_list.find_all("li")[1].get_text(strip=True).replace("Agency:", "").strip()
    else:
        agency = "N/A"
    return agency

async def scrape_enforcement_data_from(year, month):
    if year < 2013:
        print("Please choose a year >= 2013. Only enforcement actions after 2013 are available.")
        return None

    base_url = "https://oig.hhs.gov/fraud/enforcement/"
    page_number = 1
    
    titles, dates, categories, links, agencies = [], [], [], [], []
    start_date = datetime(year, month, 1).date()
    
    connector = aiohttp.TCPConnector(ssl=False)
    async with aiohttp.ClientSession(connector=connector) as session:
        while True:
            current_page_url = f"{base_url}?page={page_number}" if page_number > 1 else base_url
            print(f"Scraping page: {current_page_url}")
            
            html = await fetch_page_content(session, current_page_url)
            soup = BeautifulSoup(html, "html.parser")
            
            enforcement_entries = soup.find_all("li", class_="usa-card card--list pep-card--minimal mobile:grid-col-12")
            
            if not enforcement_entries:
                print(f"No more data found on page {page_number}. Stopping.")
                break
            
            action_tasks = []
            for entry in enforcement_entries:
                date_str = entry.find(class_="text-base-dark padding-right-105").get_text(strip=True)
                entry_date = datetime.strptime(date_str, "%B %d, %Y").date()
                
                if entry_date >= start_date:
                    title = entry.find("h2", class_="usa-card__heading").find("a").get_text(strip=True)
                    category = entry.find("ul", class_="display-inline add-list-reset").find("li").get_text(strip=True)
                    link = urljoin(base_url, entry.find("a")["href"])
                    
                    titles.append(title)
                    dates.append(date_str)
                    categories.append(category)
                    links.append(link)
                    
                    action_tasks.append(extract_agency_info(session, link))
                else:
                    agencies.extend(await asyncio.gather(*action_tasks))
                    return pd.DataFrame({
                        "Title of Enforcement Action": titles,
                        "Date": dates,
                        "Category": categories,
                        "Link": links,
                        "Agency": agencies
                    })
            
            agencies.extend(await asyncio.gather(*action_tasks))
            page_number += 1
            await asyncio.sleep(1)
    
    return pd.DataFrame({
        "Title of Enforcement Action": titles,
        "Date": dates,
        "Category": categories,
        "Link": links,
        "Agency": agencies
    })

async def main():
    year, month = 2021, 1 
    enforcement_df_Jan_2021 = await scrape_enforcement_data_from(year, month)
    print(enforcement_df_Jan_2021)
    
    # Save the DataFrame to a CSV file
    filename = f"enforcement_actions_{year}_{month:02d}.csv"
    enforcement_df_Jan_2021.to_csv(filename, index=False)
    print(f"Data saved to {filename}")
    
    return enforcement_df_Jan_2021

# Running the script
nest_asyncio.apply()
if __name__ == "__main__":
    enforcement_df_Jan_2021 = asyncio.run(main())

In [ ]:
enforcement_df_Jan_2021.head()

In [ ]:
print(f"The total number of enforcement actions in the dataframe is {enforcement_df_Jan_2021.shape[0]}.")

# Getting details of the earliest scraped enforcement action
first_entry = enforcement_df_Jan_2021.iloc[0].to_dict()
print(first_entry)

## Step 3: Plot data based on scraped data

### 1. Plot the number of enforcement actions over time (PARTNER 2)


In [ ]:
import altair as alt

# Convert 'Date' column to datetime format
enforcement_df_Jan_2021['Date'] = pd.to_datetime(enforcement_df_Jan_2021['Date'], errors='coerce')

# Aggregate the data by month and year, then count the actions
monthly_enforcement_counts = (
    enforcement_df_Jan_2021
    .groupby(enforcement_df_Jan_2021['Date'].dt.to_period('M'))
    .size()
    .reset_index(name='Action Count')
)

# Convert 'Date' to the first day of the respective month
monthly_enforcement_counts['Date'] = monthly_enforcement_counts['Date'].dt.to_timestamp()

# Generate the line chart using Altair
action_trend_chart = alt.Chart(monthly_enforcement_counts).mark_line(point=True).encode(
    x=alt.X('Date:T', title='Month-Year', axis=alt.Axis(format='%Y-%m', labelAngle=-45)),
    y=alt.Y('Action Count:Q', title='Total Actions'),
    tooltip=['Date:T', 'Action Count:Q']
).properties(
    title='Enforcement Actions Trend by Month (From January 2021)',
    width=750,
    height=400
)

action_trend_chart.display()

### 2. Plot the number of enforcement actions categorized: (PARTNER 1)

* based on "Criminal and Civil Actions" vs. "State Enforcement Agencies"


In [ ]:
# Extract Year from Date
enforcement_df_Jan_2021['Year'] = enforcement_df_Jan_2021['Date'].dt.year

# Filter data to include only relevant categories
enforcement_subset = enforcement_df_Jan_2021[
    (enforcement_df_Jan_2021['Category'] == "Criminal and Civil Actions") |
    (enforcement_df_Jan_2021['Category'] == "State Enforcement Agencies")
]

# Group the data by Month-Year and Category, and count occurrences
category_count = enforcement_subset.groupby([enforcement_subset['Date'].dt.to_period("M"), 'Category']).size().reset_index(name='Action Count')

# Standardize 'Date' column to first day of the month
category_count['Date'] = category_count['Date'].dt.to_timestamp()

# Create line chart to visualize enforcement actions by category
category_line_chart = alt.Chart(category_count).mark_line(point=True).encode(
    x=alt.X('Date:T', title='Month-Year', axis=alt.Axis(format='%Y-%m', labelAngle=-45)),
    y=alt.Y('Action Count:Q', title='Total Actions'),
    color=alt.Color('Category:N', title='Action Type'),
    tooltip=['Date:T', 'Category:N', 'Action Count:Q']
).properties(
    title='Enforcement Actions by Category Over Time',
    width=700,
    height=400
)

category_line_chart.display()


* based on five topics


In [ ]:
# Define a function to categorize actions based on title keywords
def categorize_action_by_topic(title):
    title = title.lower()
    if "health" in title:
        return "Health Care Fraud"
    elif "financial" in title or "bank" in title:
        return "Financial Fraud"
    elif "drug" in title:
        return "Drug Enforcement"
    elif "bribery" in title or "corruption" in title:
        return "Bribery/Corruption"
    else:
        return "Other"

# Apply the function to assign topics to relevant actions
enforcement_df_Jan_2021['Topic'] = enforcement_df_Jan_2021.apply(
    lambda row: categorize_action_by_topic(row['Title of Enforcement Action']) if row['Category'] == "Criminal and Civil Actions" else None, axis=1)

# Filter data to include only "Criminal and Civil Actions" category
criminal_civil_subset = enforcement_df_Jan_2021[enforcement_df_Jan_2021['Category'] == 'Criminal and Civil Actions']

# Group data by month-year and topic, counting the occurrences
topic_count = criminal_civil_subset.groupby([criminal_civil_subset['Date'].dt.to_period("M"), 'Topic']).size().reset_index(name='Action Count')

# Standardize 'Date' column to first day of the month
topic_count['Date'] = topic_count['Date'].dt.to_timestamp()

# Create line chart for topics within 'Criminal and Civil Actions'
topic_line_chart = alt.Chart(topic_count).mark_line(point=True).encode(
    x='Date:T',
    y='Action Count:Q',
    color='Topic:N',
    tooltip=['Date:T', 'Topic:N', 'Action Count:Q']
).properties(
    title="Enforcement Actions by Topic (Criminal and Civil Actions)",
    width=700,
    height=400
).interactive()

topic_line_chart.display()

## Step 4: Create maps of enforcement activity

### 1. Map by State (PARTNER 1)


In [ ]:
# Importing the required libraries
import geopandas as gpd
import matplotlib.pyplot as plt

# Filter out the enforcement actions taken by state-level agencies
state_agency_data = enforcement_df_Jan_2021[enforcement_df_Jan_2021['Agency'].str.contains("State of", na=False)]

# Clean up the state names by removing "State of" prefix
state_agency_data['State'] = state_agency_data['Agency'].str.replace("State of ", "").str.strip()

# Group the data by state and count the number of enforcement actions per state
state_action_counts = state_agency_data.groupby('State').size().reset_index(name='action_count')

# Read in the shapefile for U.S. state boundaries
shapefile_path = 'C:/Users/Shreya Work/OneDrive/Documents/GitHub/PS5/cb_2018_us_state_500k.shp'
states_map = gpd.read_file(shapefile_path).to_crs(epsg=4326)

# Merge the state-level enforcement action data with the states shapefile
states_map = states_map.merge(state_action_counts, left_on='NAME', right_on='State', how='left').fillna(0)

# Exclude non-continental U.S. states and territories from the map
excluded_states = ['AS', 'HI', 'PR', 'VI', 'MP', 'GU', 'AK']
states_map = states_map[~states_map['STUSPS'].isin(excluded_states)]

# Plot the choropleth map for the enforcement actions by state
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
states_map.plot(column='action_count', cmap='Blues', linewidth=0.8, ax=ax, edgecolor='0.8', legend=True)

# Adjust map extent to focus on the continental U.S.
ax.set_xlim([-130, -65])  # Longitude range for the continental U.S.
ax.set_ylim([24, 50])  # Latitude range for the U.S.

# Add state abbreviations or full state names to the map for clarity
for idx, row in states_map.iterrows():
    # Use the centroid of each state for label placement
    plt.text(row.geometry.centroid.x, row.geometry.centroid.y, 
             row['STUSPS'],  # Change to 'State' for full state names if preferred
             fontsize=8, ha='center', color='black')

# Add a title to the map and display it
ax.set_title('Enforcement Actions by State')
plt.show()

### 2. Map by District (PARTNER 2)


In [ ]:
# Import necessary libraries
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd

# Filter the enforcement actions related to US Attorney District-level agencies
district_data = enforcement_df_Jan_2021[enforcement_df_Jan_2021['Agency'].str.contains("District", na=False)]

# Extract and clean district names from the 'Agency' column
district_data['District'] = district_data['Agency'].str.split(',').str[1].str.strip()

# Aggregate the enforcement actions by district
district_action_counts = district_data.groupby('District').size().reset_index(name='action_count')

# Load the US Attorney District shapefile
district_shapefile_path = 'C:/Users/Shreya Work/OneDrive/Documents/GitHub/PS5/geo_export_6122d5c7-042c-4e6e-9edd-c15058235201.shp'
district_map = gpd.read_file(district_shapefile_path)

# Merge the enforcement data with the district shapefile based on judicial district names
district_shapefile.columns = district_shapefile.columns.str.strip()  # Strip any extra spaces from column names
district_map_merged = pd.merge(district_shapefile, district_action_counts, left_on='judicial_d', right_on='District', how='left').fillna(0)

# Exclude non-continental U.S. regions and territories for a more focused map
excluded_districts = ['AS', 'HI', 'PR', 'VI', 'MP', 'GU', 'AK']
district_map_merged = district_map_merged[~district_map_merged['abbr'].isin(excluded_districts)]

# Plot the choropleth for enforcement actions by district
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
district_map_merged.plot(column='action_count', cmap='Blues', linewidth=0.8, ax=ax, edgecolor='0.8', legend=True)

# Set limits for a zoomed-in view of the continental U.S.
ax.set_xlim([-130, -65])  # Longitude range for the continental U.S.
ax.set_ylim([24, 50])  # Latitude range for the U.S.

# Annotate the map with district codes or names
for idx, row in district_map_merged.iterrows():
    plt.text(row.geometry.centroid.x, row.geometry.centroid.y, 
             row['abbr'],  # Use 'District' for full district names if needed
             fontsize=8, ha='center', color='black')

# Add title and show the plot
ax.set_title('Enforcement Actions by US Attorney Districts')
plt.show()

## Extra Credit

### 1. Merge zip code shapefile with population

### 2. Conduct spatial join

### 3. Map the action ratio in each district